In [10]:
from dumbnano import dumbnano
#Reload the module
import importlib
importlib.reload(dumbnano)

<module 'dumbnano.dumbnano' from '/home/raingel/桌面/OOP_v2/dumbnano/dumbnano.py'>

In [11]:
dumb = dumbnano.NanoAmpliParser()

In [3]:
dumb.mafft_consensus("./data/Cluster_tmp/", "./data/5_assemble/")

'/home/raingel/桌面/OOP_v2/data/5_assemble'

In [5]:
dumb._exec("./dumbnano/bin/mmseqs easy-cluster ./2110.fas 2110cluster tmp --min-seq-id 0.5 -c 0.8 --cov-mode 1")

Cannot fit databases into 4G. Please use a computer with more main memory.


("Create directory tmp\neasy-cluster ./2110.fas 2110cluster tmp --min-seq-id 0.5 -c 0.8 --cov-mode 1 \n\nMMseqs Version:                     \t7e2840992948ee89dcc336522dc98a74fe0adf00\nSubstitution matrix                 \taa:blosum62.out,nucl:nucleotide.out\nSeed substitution matrix            \taa:VTML80.out,nucl:nucleotide.out\nSensitivity                         \t4\nk-mer length                        \t0\nk-score                             \tseq:2147483647,prof:2147483647\nAlphabet size                       \taa:21,nucl:5\nMax sequence length                 \t65535\nMax results per query               \t20\nSplit database                      \t0\nSplit mode                          \t2\nSplit memory limit                  \t0\nCoverage threshold                  \t0.8\nCoverage mode                       \t1\nCompositional bias                  \t1\nCompositional bias                  \t1\nDiagonal scoring                    \ttrue\nExact k-mer matching                \t0\nMa

In [31]:
import pandas as pd
import os
import edlib
df = pd.read_csv("./2023000003_barcode.tsv - 230107_barcode.tsv", sep='\t')
#Read fasta file
for f in os.scandir("./data/2_Singlebar_3/"):
    if f.is_file() and f.name.endswith(".fas"):
        sample_id = f.name[:-4]
        try:
            F = df[df['SampleID'].astype(str)==sample_id]['FwPrimer'].values[0]
            R = df[df['SampleID'].astype(str)==sample_id]['RvPrimer'].values[0]
        except IndexError:
            print(f"Sample {sample_id} not found in the barcode file, skipping")
            continue
        with open(f.path) as handle:
            with open(f"./tmp/{f.name}", "w") as output:
                for record in dumb._fasta_reader(handle):
                    #Check if the sequence is in the right orientation
                    aln_f = edlib.align(F.upper(), record['seq'].upper()[:100], mode="HW", task="locations")
                    aln_r = edlib.align(R.upper(), record['seq'].upper()[:100], mode="HW", task="locations")
                    if aln_f['editDistance'] > aln_r['editDistance']:
                        record['seq'] = dumb._reverse_complement(record['seq'])
                    output.write(f">{record['title']}\n{record['seq']}\n")

#Output: sequences with the right orientation

In [28]:
df

,SampleID,FwIndex,FwPrimer,HEAD,RvAnchor,RvIndex,RvPrimer
0,2131,cacatcaagaagttgccaat,tYtggtgYagYgattgga,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,cacatcaagaagttgccaat,GAGATGGARCARCAGGT
1,2109,agttcgttaggcggctgatt,agagtttgatymtggctcag,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,agttcgttaggcggctgatt,tacggytaccttgttacgactt
2,2110,aactcgatgattaccgatcc,atgtcaccacaaacagagactaaagc,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,aactcgatgattaccgatcc,aggggacgaccatacttgttca
3,2111,tcgtgttacaagacacgatc,atgcgatacttggtgtgaat,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,tcgtgttacaagacacgatc,cccghytgayytgrggtcdc
4,2113,cgtatgccaatccaactgtt,agagtttgatymtggctcag,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,cgtatgccaatccaactgtt,tacggytaccttgttacgactt
...,...,...,...,...,...,...,...
82,2515,tagagtctctgctgcagtct,cctngttgatyctgccagt,GCTATGCGCGAGCTGC,GCAGCTCGCGCATAGC,tagagtctctgctgcagtct,tcctgagggaaacttcg
83,2520,ggctgtggatatcgagtgca,gtcgtygtyatygghcaygt,CGCGTA,TACGCG,ggctgtggatatcgagtgca,atgacaccracrgcracrgtytg
84,2521,gatatcggacgcttgtcagc,gtcgtygtyatygghcaygt,ATGTACGCC,GGCGTACAT,gatatcggacgcttgtcagc,atgacaccracrgcracrgtytg
85,2522,gttcgaagcggagaggtatg,gtcgtygtyatygghcaygt,GACCGAGTAACGC,GCGTTACTCGGTC,gttcgaagcggagaggtatg,atgacaccracrgcracrgtytg


In [21]:
aln_f


{'editDistance': 5,
 'alphabetLength': 4,
 'locations': [(64, 88)],
 'cigar': None}

In [22]:
aln_r

{'editDistance': 9,
 'alphabetLength': 4,
 'locations': [(366, 384), (366, 385)],
 'cigar': None}

In [7]:
df[df['SampleID'].astype(str)==sample_id]['FwPrimer'].values[0]

'atgtcaccacaaacagagactaaagc'